In [6]:
pip install "pymupdf<1.23"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 154.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import fitz

def extract_links_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    links = []
    for page_num in range(len(doc)):
        page = doc[page_num]
        for link in page.get_links():
            if 'uri' in link:
                links.append(link['uri'])
    return links

In [4]:
pdf_file_path = "CIO-PDF-LINKS.pdf"
links = extract_links_from_pdf(pdf_file_path)

for idx,link in enumerate(links, start=1):
    download_and_upload(link, "thought-leadership-and-feedback-loop")
    print(f"Link {idx}: {link}")

Downloaded: NAM_Market_Recap_Mar_2025.pdf
Link 1: http://www.docs.citi.com/WealthOCIO/NAM_Market_Recap_Mar_2025.pdf
Downloaded: Investment_Strategy_Bulletin_03042025.pdf
Link 2: http://www.docs.citi.com/WealthOCIO/Investment_Strategy_Bulletin_03042025.pdf
Downloaded: ES_Investment_Strategy_Buletin_04032025.pdf
Link 3: http://www.docs.citi.com/wealthocio1/ES_Investment_Strategy_Buletin_04032025.pdf
Downloaded: DataWatch20250404.pdf
Link 4: http://www.docs.citi.com/WealthOCIO/DataWatch20250404.pdf
Downloaded: GICAAApril72025.pdf
Link 5: http://www.docs.citi.com/WealthOCIO/GICAAApril72025.pdf
Downloaded: Weekly_Insights_April_7.pdf
Link 6: http://www.docs.citi.com/WealthOCIO/Weekly_Insights_April_7.pdf
Downloaded: Global_FX_strategy_April_2025.pdf
Link 7: http://www.docs.citi.com/WealthOCIO/Global_FX_strategy_April_2025.pdf
Downloaded: NAM_Muni_Watch_April_2025.pdf
Link 8: http://www.docs.citi.com/WealthOCIO/NAM_Muni_Watch_April_2025.pdf
Downloaded: Quadrant040925.pdf
Link 9: http://www.d

In [6]:
pip install requests boto3

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import boto3
from botocore.exceptions import NoCredentialsError
from urllib.parse import urlparse
import os

def download_pdf(pdf_url):
    try:
        response = requests.get(pdf_url)
        response.raise_for_status()

        parsed_url = urlparse(pdf_url)
        file_name = os.path.basename(parsed_url.path)

        with open(file_name, 'wb') as f:
            f.write(response.content)
        
        print(f"Downloaded: {file_name}")
        return file_name
    except requests.RequestException as e:
        print(f"Error downloading PDF: {e}")
        return None

def upload_to_s3(file_name, bucket_name, s3_key=None):
    s3 = boto3.client('s3')
    if s3_key is None:
        s3_key = file_name

    try:
        s3.upload_file(file_name, bucket_name, s3_key)
        # print(f"Uploaded to s3://{thought-leadership-and-feedback-loop}/{s3_key}")
    except FileNotFoundError:
        print("The file was not found")
    except NoCredentialsError:
        print("AWS credentials not available")

def download_and_upload(pdf_url, bucket_name, s3_key=None):
    file_name = download_pdf(pdf_url)
    if file_name:
        upload_to_s3(file_name, bucket_name, s3_key)
        os.remove(file_name)  # Clean up local file

# Example usage
# pdf_url = "https://www.docs.citi.com/WealthOCIO/NAM_Market_Recap_Mar_2025.pdf"
# bucket_name = "your-s3-bucket-name"
# s3_key = "uploads/sample.pdf"  # Optional: change destination path in bucket

# download_and_upload(pdf_url, "thought-leadership-and-feedback-loop")
